# Fase 2

In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  


In [56]:
# Trasladar todas las tuplas del diccionario a una sola lista
# Paso 1: guardar datos de cada extracción en variables 

extracion_maira = pd.read_json('CSV_JSON/datos_filtrados_2010_2024.json')
extracion_yami = pd.read_json('CSV_JSON/datos_filtrados_Yami_2010_2024.json')
extracion_carolina = pd.read_json('CSV_JSON/datos_filtrados_2010_2024_carolina.json')
extracion_laura = pd.read_json('CSV_JSON/datos_filtrados_2010_2024_laura.json')


In [57]:
# Paso 2: Creamos una función para iterar por cada variable para guardar los datos en listas de tuplas

def itera_variables(extracion):
    anios = [i for i in range(2010, 2025)]
    generos = ['Action', 'Comedy', 'Drama']
    resultados_tuplas = []

    for anio in anios:
        for genero in generos:
            for tupla in extracion[anio][genero]:
                resultados_tuplas.append(tuple(tupla))

    return resultados_tuplas

In [58]:
# Sumamos todas las lista en una
resultado_tuplas_total = itera_variables(extracion_maira) + itera_variables(extracion_yami) + itera_variables(extracion_carolina) + itera_variables(extracion_laura)

In [61]:
len(resultado_tuplas_total)

17395

In [60]:
resultado_tuplas_total[:10]

[('Movie', 'Tekken', 2010, 3, 'tt0411951'),
 ('Movie', 'Venus & Vegas', 2010, 11, 'tt0423474'),
 ('Movie', 'The A-Team', 2010, 6, 'tt0429493'),
 ('Movie', 'Segurança Nacional', 2010, 5, 'tt0439801'),
 ('Movie', 'Scott Pilgrim vs. the World', 2010, 11, 'tt0446029'),
 ('Movie', 'Taken by Force', 2010, None, 'tt0465637'),
 ('Movie', 'Guns, Drugs and Dirty Money', 2011, None, 'tt0464032'),
 ('Movie', 'Komaram Puli', 2010, 9, 'tt0464162'),
 ('Movie', 'Prince of Persia: The Sands of Time', 2010, 5, 'tt0473075'),
 ('Movie', 'Unstoppable', 2010, 12, 'tt0477080')]

In [62]:
# Verificar que no hay elementos repetidos
# Crear un conjunto vacío
elementos_vistos = set()

# Verificar duplicados
contador = 0
for elemento in resultado_tuplas_total:
    if elemento in elementos_vistos:
        contador += 1
        resultado_tuplas_total.remove(elemento)
    elementos_vistos.add(elemento)
    
print(f"Se ha encontrado {contador} elementos repetidos")
print(f"Tras remover las tuplas repetidas, nuestra lista se quedó con {len(resultado_tuplas_total)} elementos.")

Se ha encontrado 2401 elementos repetidos
Tras remover las tuplas repetidas, nuestra lista se quedó con 14994 elementos.


In [66]:
# Guardar lista final de manera permanente
df_final = pd.DataFrame(resultado_tuplas_total)
df_final[2] = df_final[2].astype('Int64')
df_final[3] = df_final[3].astype('Int64')
df_final.to_csv('resultado_tuplas_total.csv', index=False)
df_final.to_json('resultado_tuplas_total.json')

In [8]:
resultado_tuplas_total = pd.read_csv('resultado_tuplas_total.csv', header=None) # Ejecutar a partir de aquí

In [9]:
resultado_tuplas_total = [tuple(fila) for fila in resultado_tuplas_total.values]
resultado_tuplas_total = resultado_tuplas_total[1:]

In [10]:
len(resultado_tuplas_total)

14994

In [11]:
url_imdb = "https://www.imdb.com/"
url_rotten = "https://www.rottentomatoes.com/"
url_google = 'https://www.google.com/'

In [17]:
for peli in resultado_tuplas_total[100:]:
    id_imdb = peli[-1]
    nombre = peli[1]
    print(id_imdb)
    print(nombre)
    break

tt1228705
Iron Man 2


In [18]:
search_tomato = id_imdb + ' ' + nombre + ' ' + 'tomatometer'
search_tomato

'tt1228705 Iron Man 2 tomatometer'

In [22]:
resultados_fase2 = []
not_found = []

In [23]:
# Inicializa el navegador Chrome y maximiza la ventana
driver1 = webdriver.Chrome()
sleep(3)
driver1.maximize_window()

# Navega a la página web "https://www.imdb.com/"
driver1.get(url_imdb)
sleep(3)

# Acepta las cookies haciendo clic en el elemento con el selector CSS "#didomi-notice-agree-button"
driver1.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
sleep(3)

# Inicializa el navegador Chrome y maximiza la ventana
driver2 = webdriver.Chrome()
sleep(3)
driver2.maximize_window()

# Navega al buscador
driver2.get(url_google)
sleep(3)

# Acepta las cookies haciendo clic
driver2.find_element("css selector", "#L2AGLb > div").click()
sleep(3)

contador = 0
on_off = True

'''
Maíra: 0:50, 200:400 ok, next = 400:1000
Yami: 50:100
Laura: 100:150
Carolina: 150:200
'''

# Iterar por los ids_imdbs para pasarlos por el campo search:
for peli in resultado_tuplas_total[100:150]:
    id_imdb = peli[-1]
    nombre = peli[1]

    driver1.find_element("css selector", "#suggestion-search").send_keys(id_imdb, Keys.ENTER)
    sleep(3)

    try:
        imdb_rating = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-3a4309f8-0.bjXIAP.sc-1f50b7c-1.bfrqUc > div > div:nth-child(1) > a > span > div > div.sc-eb51e184-0.kFvAju > div.sc-eb51e184-2.czkfBq > span.sc-eb51e184-1.cxhhrI").text
        sleep(2)
    except Exception as e:
        imdb_rating = None
        error = f"No hay imdb_rating para la peli {peli[1]} con id_imdb {peli[-1]}, error de tipo: {e}."
        not_found.append(error)

    try:                                                  
        direccion = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(1) > div > ul > li > a").text
        sleep(2)
    except:
        try:
            direccion = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(1) > div > ul > li > a").text
            sleep(2)
        except Exception as e:
            direccion = None
            error = f"No hay director(es) para la peli {peli[1]} con id_imdb {peli[-1]}, error de tipo: {e}."
            not_found.append(error)

    try:                                                  
        guionista = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(2) > div > ul > li > a").text
        sleep(2)
    except:
        try:
            guionista = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(2) > div > ul").text
            sleep(2)
        except Exception as e:
            guionista = None
            error = f"No hay guionista para la peli {peli[1]} con id_imdb {peli[-1]}, error de tipo: {e}."
            not_found.append(error)

    try:                                                  
        argumento = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > p > span.sc-2d37a7c7-2.PeLXr").text
        sleep(2)
    except:
        try:
            argumento = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > p > span.sc-2d37a7c7-2.PeLXr").text
            sleep(2)
        except Exception as e:
            argumento = None
            error = f"No hay argumento para la peli {peli[1]} con id_imdb {peli[-1]}, error de tipo: {e}."
            not_found.append(error)

    try:                                                 
        duracion = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-1f50b7c-0.PUxFE > ul > li:nth-child(3)").text
        sleep(2)
    except:
        try: 
            duracion = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-1f50b7c-0.PUxFE > ul > li:nth-child(2)").text
            sleep(2)  
        except Exception as e:  
            duracion = None
            error = f"No hay duracion para la peli {peli[1]} con id_imdb {peli[-1]}, error de tipo: {e}."
            not_found.append(error)

    try:
        nombre = driver1.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-1f50b7c-0.PUxFE > h1 > span").text
        sleep(2)
    except Exception as e:
        nombre = None
        error = f"No hay nombre para la peli {peli[1]} con id_imdb {peli[-1]}, error de tipo: {e}."
        not_found.append(error)


    search_tomato = id_imdb + ' ' + nombre + ' ' + 'tomatometer'

    driver2.find_element("css selector", "#APjFqb").send_keys(search_tomato, Keys.ENTER) # Paso el string al buscador de google
    sleep(5)


    try:                                
        sleep(5)                              #rso > div:nth-child(1) > div > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a > h3
        driver2.find_element("css selector", "#rso > div:nth-child(1) > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a > h3").click() # Pincho en el primer resultado de la búsqueda de google
        sleep(4)

        #if on_off:
        #    driver2.find_element("css selector", "#onetrust-accept-btn-handler").click() # Acepta cookies
        #    on_off = False
        #   sleep(2)     

        try:                                                   
            tomatometer = driver2.find_element("css selector", "#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button:nth-child(3) > rt-text").text # Extrae el Rating tomatometer
            sleep(3)
        
        except Exception as e:
            tomatometer = None
            error = f"No hay tomatometer para la peli {peli[1]} con id_imdb {peli[-1]}, error de tipo: {e}."
            not_found.append(error)

        driver2.back()
        sleep(3)

        driver2.back()
        sleep(3)
                            
    except:
        try:                                      #rso > div:nth-child(1) > div > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a > h3
            sleep(5)
            driver2.find_element("css selector", "#rso > div.MjjYud > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a > h3").click() # Pincho en el primer resultado de la búsqueda de google
            sleep(4)

            #if on_off:
            #   driver2.find_element("css selector", "#onetrust-accept-btn-handler").click() # Acepta cookies
            #   on_off = False
            #   sleep(2) 

            try:                                                   
                tomatometer = driver2.find_element("css selector", "#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button:nth-child(3) > rt-text").text # Extrae el Rating tomatometer
                sleep(3)
            
            except Exception as e:
                tomatometer = None
                error = f"No hay tomatometer para la peli {peli[1]} con id_imdb {peli[-1]}, error de tipo: {e}."
                not_found.append(error)

            driver2.back()
            sleep(3)

            driver2.back()
            sleep(3)

        except Exception as e:
            tomatometer = None
            error = f"No hay tomatometer para la peli {peli[1]} con id_imdb {peli[-1]}, error de tipo: {e}."
            not_found.append(error)
            
            driver2.back()
            sleep(3)


    tupla = imdb_rating, tomatometer, direccion, guionista, argumento, duracion, nombre        

    if None in tupla:
        contador += 1
        print(f"La tupla de la peli {nombre} no está completa: {tupla} Vamos por la iteración: {contador}")
    else:
        resultados_fase2.append(tupla)
        contador += 1
        print(f"Hemos añadido la peli {nombre} a la lista. Vamos por la iteración: {contador}.")

driver1.close()
driver2.close()

La tupla de la peli Iron Man 2 no está completa: ('6,9', None, 'Jon Favreau', 'Justin Theroux', 'Con el mundo sabiendo que es Iron Man, Tony Stark debe lidiar tanto con su salud en declive como con un loco vengativo con vínculos con el legado de su padre.', '2h 4min', 'Iron Man 2') Vamos por la iteración: 1
La tupla de la peli Downstream no está completa: ('3,6', None, 'Simone Bartesaghi', 'Philip KimNeil Kinsella', 'El futuro de Estados Unidos, cuando el mundo se quede sin petróleo.', '1h 41min', 'Downstream') Vamos por la iteración: 2
La tupla de la peli Collide no está completa: ('5,9', '24%', 'June Daguiso', 'June Daguiso', '', None, 'Collide') Vamos por la iteración: 3
La tupla de la peli The Yellow Sea no está completa: ('7,3', None, 'Na Hong-jin', 'Won-Chan HongNa Hong-jin', 'Cuando el intento de matar a un profesor sale mal, se desencadenan una serie de eventos violentos que obligan al taxista a correr por su vida.', '2h 37min', 'The Yellow Sea') Vamos por la iteración: 4
La tu

KeyboardInterrupt: 

In [147]:
len(resultados_fase2)

73

In [145]:
# Se tarda aproximadamente 33 minutos para extraer 50 peliculas.
# Se tarda aproximadamente 67 minutos para extraer 100 peliculas.

In [139]:
# not_found.clear()
# resultados_fase2.clear()

In [148]:
# Guardar los datos de manera permanente
df_resultados = pd.DataFrame(resultados_fase2)
df_errores = pd.DataFrame(not_found)
df_resultados.to_csv('resultados_fase2_maira_0-50_200-400.csv') # Cambiar nombre !!! 
df_resultados.to_json('resultados_fase2_maira_0-50_200-400.json')
df_errores.to_csv('errores_fase2_maira_0-50_200-400.csv')
df_errores.to_json('errores_fase2_maira_0-50_200-400.json')

2 - Para sacar datos del Rotten Tomatoes, hacerlo directamente desde el buscador o buscar otra página que sea mas sencilla de extraer que el propio Rotten.

3 - Añadir el genero en Fase 2 es opcional: Lo añadimos. 

4 - La conversion de horas a minutos no es obligatoria: Añadir tal cuál está sin convertir 

5 - En los try except, meter mas selectores: el código actual tiene el objetivo de capturar errores. 

6 - Queremos la info en Inglés o en Español?

7 - Convertir todo el notebook a funciones.

In [ ]:
# Puntuacion de IMDB (en caso de que la tenga).

# Puntuación de Rotten Tomatoes (Tomatometer).

# Dirección (director/a o directore/as de cada película).

# Gionistas (de cada película).

# Argumento.

# Duración. >> Tal cuál está 

# Nombre de la película

# Genero !!! 

# NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. Siguiendo el siguiente ejemplo:

# [(7.7, 77, "Richard Donner", ["Chris ColumbusSteven", "Spielberg"], "Los Goonies son un grupo de amigos que viven en Goon Docks, Astoria, pero sus casas han sido compradas y van a ser demolidas. Sin embargo, vivirán su última aventura en busca de un tesoro que pueda salvar el barrio.", "Aventura", "1h 54min", "Los Gonnies"), ...]